In [1]:
import cv2
import os
import numpy as np
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from skimage.morphology import skeletonize
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025")
import ani


In [2]:
def estimate_binary_height(binary, th_low, th_high):
    labeled = label(binary)
    heights = [p.bbox[2] - p.bbox[0]
               for p in regionprops(labeled)
               if th_low <= (p.bbox[2] - p.bbox[0]) <= th_high]
    if not heights:
        return None
    mu, sigma = np.mean(heights), np.std(heights)
    return int(mu - 0.5 * sigma), int(mu + 0.5 * sigma)

def load_and_preprocess_image(image_path):
    gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, bin_img = cv2.threshold(gray, 0, 255,
                              cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    dilated = cv2.dilate(bin_img, kernel, iterations=1)
    return np.bitwise_not(dilated ),dilated

def random_label_cmap(num_labels, seed=25):
    np.random.seed(seed)
    colors = np.random.rand(num_labels, 3)
    colors = np.vstack([[0, 0, 0], colors])
    return mcolors.ListedColormap(colors)

def filter_lines(gray, scales, eta):
    responses = []
    for scale in scales:
        raw = ani.anigauss(
            gray.astype(np.float64), sigv=scale, sigu=eta * scale, phi=0, derv=2, deru=0)
        responses.append(raw.flatten())
    max_resp = np.max(np.array(responses), axis=0).reshape(gray.shape)
    thr = np.mean(max_resp) + 0.4 * np.std(max_resp)
    return (max_resp > thr).astype(np.uint8) * 255

def compute_up_down_distances(binary_image, d):

    binary = (binary_image == 0).astype(np.uint8)
    h, w = binary.shape

    up_dist = np.full((h, w), d, dtype=np.uint16)
    down_dist = np.full((h, w), d, dtype=np.uint16)

    for i in range(1, d + 1):
        shifted = np.zeros_like(binary)
        shifted[i:, :] = binary[:-i, :]
        mask = (binary == 0) & (shifted == 1)
        up_dist[mask] = np.minimum(up_dist[mask], i)
    for i in range(1, d + 1):
        shifted = np.zeros_like(binary)
        shifted[:-i, :] = binary[i:, :]
        mask = (binary == 0) & (shifted == 1)
        down_dist[mask] = np.minimum(down_dist[mask], i)

    total_vertical_distance = up_dist + down_dist
    binary_mask = np.where((binary_image == 255) & (total_vertical_distance>d), 255, 0).astype(np.uint8)
    return binary_mask

In [3]:
input_folders = [
    "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/Syr341/img-Syr341/validation/"
]

binary_output_root = "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/Syr341/text-line-prediction-Syr341/validation/"
color_output_root = "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/Syr341/text-line-prediction-Syr341/colored_validation/"

for folder in input_folders:
    for root, _, files in os.walk(folder):
        for file in files:
            if not file.lower().endswith(".jpg"):
                continue

            print(f"Processing: {file}")
            image_path = os.path.join(root, file)

            # Construct a mirrored output path
            relative_path = os.path.relpath(root, start=input_folders[0])
            binary_out_dir = os.path.join(binary_output_root, relative_path)
            color_out_dir = os.path.join(color_output_root, relative_path)
            os.makedirs(binary_out_dir, exist_ok=True)
            os.makedirs(color_out_dir, exist_ok=True)
            out_path_color = os.path.join(color_out_dir, os.path.splitext(file)[0] + ".png")
            out_path_binary = os.path.join(binary_out_dir, os.path.splitext(file)[0] + ".png")

            # 1) Load and preprocess
            bin, inv_bin = load_and_preprocess_image(image_path)

            # 2) Estimate character height range
            th_low = bin.shape[0] * 0.001
            th_high = bin.shape[0] * 0.1
            height_range = estimate_binary_height(inv_bin, th_low, th_high)

            # 3) Generate line masks
            scales = [height_range[0], height_range[1]]
            lines_mask = filter_lines(bin, scales, eta=2)
            v_separators = compute_up_down_distances(255-lines_mask, 10 * height_range[0])
            inverted_v_separators = cv2.bitwise_not(v_separators)
            lines_mask_split = cv2.bitwise_and(lines_mask, inverted_v_separators)

            # 4) Detect center of lines
            bw = lines_mask_split > 0
            lines_mask_skel = skeletonize(bw).astype(np.uint8) * 255
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 5))
            lines_mask_thick = cv2.dilate(lines_mask_skel, kernel, iterations=1)

            # Filter short blob lines
            num_lab, labels, stats, _ = cv2.connectedComponentsWithStats(lines_mask_thick, connectivity=8)
            lines = np.zeros_like(lines_mask_skel)
            for lbl in range(1, num_lab):
                width = stats[lbl, cv2.CC_STAT_WIDTH]
                height = stats[lbl, cv2.CC_STAT_HEIGHT]
                if width >= 20*height_range[0] and height <= 10*height_range[0]:
                    lines[labels == lbl] = 255

            # 5) Dilate lines region
            d = 2 * height_range[0]
            kern_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*d+1, 2*d+1))
            dilated = cv2.dilate(lines.astype(np.uint8), kern_ellipse)

            # 6) Final intersection with binary image
            final_mask = (((dilated > 0) & (inv_bin > 0)) * 255).astype(np.uint8)

            # 7) Watershed segmentation
            lines_bool = lines > 0
            markers = label(lines_bool)
            distance = ndi.distance_transform_edt(lines_bool)
            final_mask_bool = final_mask > 0
            combined = lines_bool | final_mask_bool
            segmented_lines = watershed(image=distance, markers=markers, mask=combined, watershed_line=True, connectivity=2)

            # Use a random label colormap
            num_labels = int(segmented_lines.max()) + 1
            cmap = random_label_cmap(num_labels, seed=25)

            # Save color output
            plt.imsave(out_path_color, segmented_lines, cmap=cmap)

            # Save binary output
            cv2.imwrite(out_path_binary, (segmented_lines > 0).astype(np.uint8) * 255)


Processing: 025.jpg
Processing: 428.jpg
Processing: 143.jpg
Processing: 166.jpg
Processing: 133.jpg
Processing: 278.jpg
Processing: 338.jpg
Processing: 198.jpg
Processing: 328.jpg
Processing: 039.jpg


In [4]:
"""
@author: Silvia Zottin
Evaluation code for FEST 2025: few shot text line segmentation ICDAR 2025 competition.

"""

from sklearn.metrics import precision_score, recall_score
import os
import os.path
import numpy as np
import cv2


def get_mask(path, filename, binarize=True):
    # retrieving the corresponding mask based on the input path and filename
    mask_path = os.path.join(path, filename)
    mask_path = os.path.splitext(mask_path)[0] + ".png"
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # if the flag is true binarize the image before returning it
    if binarize:
        _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    return mask


def calculate_IoU(component1, component2):
    intersection = np.logical_and(component1, component2).sum()
    union = np.logical_or(component1, component2).sum()
    if union == 0:
        return 0
    return intersection / union


def find_best_matches(gt_components, pred_components, num_gt, num_pred):
    matches = []
    matches_all = []
    for gt_label in range(1, num_gt):
        best_iou = 0
        best_match = -1
        gt_mask = (gt_components == gt_label)

        for pred_label in range(1, num_pred):
            pred_mask = (pred_components == pred_label)
            iou = calculate_IoU(gt_mask, pred_mask)

            if iou > best_iou:
                best_iou = iou
                best_match = pred_label

            if iou >= 0.75:
                matches_all.append((gt_label, pred_label))

        if best_match != -1:
            matches.append((gt_label, best_match))

    return matches, matches_all


def calculate_pixel_and_line_IU(gt_components, pred_components, matches, threshold=0.75):
    TP, FP, FN = 0, 0, 0
    CL, ML, EL = 0, 0, 0

    for gt_label, pred_label in matches:
        gt_mask = (gt_components == gt_label)
        pred_mask = (pred_components == pred_label)

        tp = np.logical_and(gt_mask, pred_mask).sum()
        fp = np.logical_and(np.logical_not(gt_mask), pred_mask).sum()
        fn = np.logical_and(gt_mask, np.logical_not(pred_mask)).sum()

        TP += tp
        FP += fp
        FN += fn

        precision = precision_score(np.array(gt_mask).flatten(), np.array(pred_mask).flatten(), zero_division=0)
        recall = recall_score(np.array(gt_mask).flatten(), np.array(pred_mask).flatten(), zero_division=0)


        if precision >= threshold and recall >= threshold:
            CL += 1
        elif recall < threshold:
            ML += 1
        elif precision < threshold:
            EL += 1

    pixel_IU = 0 if (TP + FP + FN) == 0 else TP / (TP + FP + FN)
    line_IU = 0 if (CL + ML + EL) == 0 else CL / (CL + ML + EL)
    print("TP: ", TP)
    print("FP: ", FP)
    print("FN: ", FN)
    print("pixel_IU: ", pixel_IU)
    print("CL: ", CL)
    print("ML: ", ML)
    print("EL: ", EL)
    print("line_IU: ", line_IU)

    return pixel_IU, line_IU


def evaluate_metrics(gt_img, pred_img):
    num_gt, gt_components = cv2.connectedComponents(gt_img)
    num_pred, pred_components = cv2.connectedComponents(pred_img)

    matches, matches_all = find_best_matches(gt_components, pred_components, num_gt, num_pred)
    pixel_IU, line_IU = calculate_pixel_and_line_IU(gt_components, pred_components, matches)

    N1 = num_gt - 1
    N2 = num_pred - 1

    M = len(matches_all)

    DR = 0 if N1 == 0 else M / N1
    RA = 0 if N1 == 0 else M / N2
    FM = 0 if DR + RA == 0 else 2 * (DR * RA) / (DR + RA)

    return pixel_IU, line_IU, DR, RA, FM


def udiads_textline_evaluate(result_directory, gt_directory):
    """
    Evaluate the results provided by the files in result_directory with respect
    to the ground truth information given by the files in gt_directory.
    """

    # Check whether result_directory and gt_directory are directories
    if not os.path.isdir(result_directory):
        print("The result folder is not a directory")
        return

    if not os.path.isdir(gt_directory):
        print("The gt folder is not a directory")
        return

    pixel_list = []
    line_list = []
    DR_list = []
    RA_list = []
    FM_list = []
    # For each file of the ground truth directory read the result
    for f in sorted(os.listdir(gt_directory)):
        # retrieving binarized gt and predicted mask
        segmentation_bin = get_mask(result_directory, f)
        ground_truth_bin = get_mask(gt_directory, f)

        # calculating IoU based metrics
        print("Calculating metrics...")
        pixel_IU, line_IU, DR, RA, FM = evaluate_metrics(ground_truth_bin, segmentation_bin)
        pixel_list.append(pixel_IU)
        line_list.append(line_IU)
        DR_list.append(DR)
        RA_list.append(RA)
        FM_list.append(FM)

        # print per‑page results
        print(
            f"[{f}] "
            f"Pixel IU: {pixel_IU:.4f}, "
            f"Line IU: {line_IU:.4f}, "
            f"DR: {DR:.4f}, "
            f"RA: {RA:.4f}, "
            f"F‑measure: {FM:.4f}"
        )

    return np.mean(pixel_list), np.mean(line_list), np.mean(DR_list), np.mean(RA_list), np.mean(FM_list)




In [5]:
Pixel_IU, Line_IU, DR, RA, FM = udiads_textline_evaluate(result_directory="/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/Syr341/text-line-prediction-Syr341/validation",
                                                         gt_directory="/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/Syr341/text-line-gt-Syr341/validation")

print("Pixel IU: ", Pixel_IU)
print("Line IU: ", Line_IU)
print("Detection Rate: ", DR)
print("Recognition Accuracy: ", RA)
print("F-measure: ", FM)

Calculating metrics...
TP:  401722
FP:  70113
FN:  164520
pixel_IU:  0.6312859960242317
CL:  96
ML:  79
EL:  6
line_IU:  0.5303867403314917
[025.png] Pixel IU: 0.6313, Line IU: 0.5304, DR: 0.2253, RA: 0.1475, F‑measure: 0.1783
Calculating metrics...
TP:  136865
FP:  83811
FN:  94770
pixel_IU:  0.4338777476969117
CL:  18
ML:  47
EL:  3
line_IU:  0.2647058823529412
[039.png] Pixel IU: 0.4339, Line IU: 0.2647, DR: 0.0686, RA: 0.2400, F‑measure: 0.1067
Calculating metrics...
TP:  490147
FP:  96977
FN:  90698
pixel_IU:  0.723120524267433
CL:  148
ML:  32
EL:  0
line_IU:  0.8222222222222222
[133.png] Pixel IU: 0.7231, Line IU: 0.8222, DR: 0.6464, RA: 0.6500, F‑measure: 0.6482
Calculating metrics...
TP:  472295
FP:  94767
FN:  160660
pixel_IU:  0.6490047023451263
CL:  120
ML:  57
EL:  7
line_IU:  0.6521739130434783
[143.png] Pixel IU: 0.6490, Line IU: 0.6522, DR: 0.2270, RA: 0.2258, F‑measure: 0.2264
Calculating metrics...
TP:  496794
FP:  89217
FN:  108094
pixel_IU:  0.7157332103932402
CL:  